In [ ]:
!pip install -q diffusers transformers accelerate wandb

In [ ]:
import torch
from diffusers import DiffusionPipeline

import wandb
from wandb.integration.diffusers import autolog

In [ ]:
# Define the Stable Diffusion XL pipeline
pipeline = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16
)

# Set the device for the pipeline
pipeline = pipeline.to("cuda")

pipe.enable_sequential_cpu_offload()

# Reduce memeory footprint
# pipeline.enable_xformers_memory_efficient_attention()

In [ ]:
# Set the prompt
prompt = "A close up portrait of an old village woman, white hair, olive green eyes, dramatic light, dreamlike light, photorealism, ultra detail, 85mm lens, indian village backdrop, long exposure background"   # @param {type:"string"}

# Set the negative prompt, or leave it `None` if you don't want to use it
negative_prompt = None  # @param {type:"raw"}

# Set the height and width of the generated image
height, witdth = 1024, 1024

# Automatically generate a seed for the generation, optionally give one of your own.
# This would ensure reproducibility of the experiment.
seed = 42 # @param {type:"raw"}
generator = torch.Generator(device="cuda").manual_seed(seed)

# Additional configs for the experiment
configs = {
    "eta": 0.0,
    "guidance_rescale": 0.0,
    "seed": seed,
}

In [ ]:
autolog()

image = pipeline(
    prompt,
    negative_prompt=negative_prompt,
    generator=generator,
    height=height,
    width=witdth,
    callback=callback,
    eta=configs["eta"],
    guidance_rescale=configs["guidance_rescale"],
)

wandb.finish()